<a href="https://colab.research.google.com/github/BrainyHaron/DigitalBreakthrough/blob/Brainy's_Projects/das_pirat_OS_vol_3(LSI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
!pip install nltk
!pip install scipy
from nltk.stem import *
from nltk.stem.porter import *
stemmer = PorterStemmer()
import os
import json
import pandas as pd
from google.colab import drive
# из библиотеки google.colab импортируем класс files
from google.colab import files
import pandas as pd
import math

In [38]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
os.chdir("/content/drive/MyDrive/das_pirat_OS/labeled")
content = os.listdir()

In [40]:
"docs = '/content/drive/MyDrive/das_pirat_OS/labeled'"

"docs = '/content/drive/MyDrive/das_pirat_OS/labeled'"

In [41]:
class LSI(object):
	def __init__(self, stopwords, ignorechars, docs):
		self.wdict = {}
		self.dictionary = []
		self.docs = []
		self.stopwords = stopwords
		if type(ignorechars) == str: ignorechars = ignorechars.encode('utf-8')
		self.ignorechars = ignorechars
		for doc in docs: self.add_doc(doc)

	def prepare(self):
		self.build()
		self.calc()

	def dic(self, word, add = False):
		if type(word) == str: word = word.encode('utf-8')
		# чистим от лишних символом
		word = word.lower().translate(None, self.ignorechars)
		word = word.decode('utf-8')
		# приводим к начальной форме
		word = stemmer.stem(word)
		# если слово есть в словаре возвращаем его номер
		if word in self.dictionary: return self.dictionary.index(word)
		else:
			# если нет и стоит флаг автоматически добавлять то пополняем словари возвращвем код слова
			if add:
				#self.ready = False
				self.dictionary.append(word)
				return len(self.dictionary) - 1
			else:
				return None


	def add_doc(self, doc):
		words = [self.dic(word, True) for word in doc.lower().split()]
		self.docs.append(words)
		for word in words:
			if word in self.stopwords:
				continue
			elif word in self.wdict:
				self.wdict[word].append(len(self.docs) - 1)
			else:
				self.wdict[word] = [len(self.docs) - 1]

	def build(self):
      # убираем одиночные слова
		self.keys = [k for k in self.wdict.keys() if len(self.wdict[k]) > 0]
		self.keys.sort()
      # создаём пустую матрицу
		self.A = np.zeros([len(self.keys), len(self.docs)])
      # наполняем эту матрицу
		for i, k in enumerate(self.keys):
			for d in self.wdict[k]:
				self.A[i,d] += 1

#Вычисление U, S Vt - матриц
	def calc(self):
		self.U, self.S, self.Vt = np.linalg.svd(self.A)


	def TFIDF(self):
      # всего кол-во слов на документ
		wordsPerDoc = sum(self.A, axis=0)
      # сколько документов приходится на слово
		docsPerWord = sum(np.asarray(self.A > 0, 'i'), axis=1)
		rows, cols = self.A.shape
		for i in range(rows):
			for j in range(cols):
				self.A[i,j] = (self.A[i,j] / wordsPerDoc[j]) * np.log(float(cols) / docsPerWord[i])

	def dump_src(self):
		self.prepare()
		print ('Здесь представлен расчет матрицы ')
		for i, row in enumerate(self.A):
			print (self.dictionary[i], row)

	def print_svd(self):
		self.prepare()
		print ('Здесь сингулярные значения')
		print (self.S)
		print ('Здесь первые 3 колонки U матрица ')
		for i, row in enumerate(self.U):
			print (self.dictionary[self.keys[i]], row[0:3])
		print ('Здесь первые 3 строчки Vt матрица')
		print (-1*self.Vt[0:3, :])

	def find(self, word):
		self.prepare()
		idx = self.dic(word)
		if not idx:
			print('слово не встречается')
			return []
		if not idx in self.keys:
			print('слово отброшено как не имеющее значения которое через stopwords')
			return []
		idx = self.keys.index(idx)
		print('word --- '), word, '=', self.dictionary[self.keys[idx]], '.\n'
		# получаем координаты слова
		wx, wy = (-1 * self.U[:, 1:3])[idx]
		print(('word{}\t{:0.2f}\t{:0.2f}\t{}\n').format(idx, wx, wy, word)) #:0.2f
		arts = []
		xx, yy = -1 * self.Vt[1:3, :]
		for k, v in enumerate(self.docs):
			# получаем координаты документа
			ax, ay = xx[k], yy[k]
			#вычисляем расстояние между словом и документом
			dx, dy = float(wx - ax), float(wy - ay)
			arts.append((k, v, ax, ay, np.sqrt(dx * dx + dy * dy)))
		# возвращаем отсортированный по расстоянию список
		return sorted(arts, key = lambda a: a[4])


In [44]:

#path = '/content/drive/MyDrive/das_pirat_OS/labeled'
#full_list3=[]

docs =[
	"Британская полиция знает о местонахождении основателя WikiLeaks",
	"В суде США США начинается процесс против россиянина, рассылавшего спам",
	"Церемонию вручения Нобелевской премии мира бойкотируют 19 стран",
	"В Великобритании арестован основатель сайта Wikileaks Джулиан Ассандж",
	"Украина игнорирует церемонию вручения Нобелевской премии",
	"Шведский суд отказался рассматривать апелляцию основателя Wikileaks",
	"НАТО и США разработали планы обороны стран Балтии против России",
	"Полиция Великобритании нашла основателя WikiLeaks, но, не арестовала",
	"В Стокгольме и Осло сегодня состоится вручение Нобелевских премий"
]
#word = "США"
ignorechars = ''',:'!'''
lsa = LSI([], ignorechars, docs)
lsa.build()
lsa.dump_src()
lsa.calc()
lsa.print_svd()

for res in lsa.find(word):
	print (res[0], res[4], res[1], docs[res[0]])

Здесь представлен расчет матрицы 
британская [1. 0. 0. 0. 0. 0. 0. 0. 0.]
полиция [1. 0. 0. 0. 0. 0. 0. 1. 0.]
знает [1. 0. 0. 0. 0. 0. 0. 0. 0.]
о [1. 0. 0. 0. 0. 0. 0. 0. 0.]
местонахождении [1. 0. 0. 0. 0. 0. 0. 0. 0.]
основателя [1. 0. 0. 0. 0. 1. 0. 1. 0.]
wikileak [1. 0. 0. 1. 0. 1. 0. 1. 0.]
в [0. 1. 0. 1. 0. 0. 0. 0. 1.]
суде [0. 1. 0. 0. 0. 0. 0. 0. 0.]
сша [0. 2. 0. 0. 0. 0. 1. 0. 0.]
начинается [0. 1. 0. 0. 0. 0. 0. 0. 0.]
процесс [0. 1. 0. 0. 0. 0. 0. 0. 0.]
против [0. 1. 0. 0. 0. 0. 1. 0. 0.]
россиянина [0. 1. 0. 0. 0. 0. 0. 0. 0.]
рассылавшего [0. 1. 0. 0. 0. 0. 0. 0. 0.]
спам [0. 1. 0. 0. 0. 0. 0. 0. 0.]
церемонию [0. 0. 1. 0. 1. 0. 0. 0. 0.]
вручения [0. 0. 1. 0. 1. 0. 0. 0. 0.]
нобелевской [0. 0. 1. 0. 1. 0. 0. 0. 0.]
премии [0. 0. 1. 0. 1. 0. 0. 0. 0.]
мира [0. 0. 1. 0. 0. 0. 0. 0. 0.]
бойкотируют [0. 0. 1. 0. 0. 0. 0. 0. 0.]
19 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
стран [0. 0. 1. 0. 0. 0. 1. 0. 0.]
великобритании [0. 0. 0. 1. 0. 0. 0. 1. 0.]
арестован [0. 0. 0. 1. 0. 0. 0. 0